In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [43]:
import sys
import base64
from collections import defaultdict

In [5]:
from antispam.html_parser import SpamHTMLParser
from antispam.feature_extractor import HTMLDataExtractor, HTMLFeatureExtractor

In [6]:
DATA_FILE  = './data/train-set-ru-b64-utf-8.txt'

In [ ]:
# tags_global_stats - статистика об общем кол-ве использований тегов
# tag_stats - статистикао о встречаемости тега

In [ ]:


tag_stats = {
    "0": defaultdict(int),
    "1": defaultdict(int)
}

def collect_tags_stats(label, data):
    for key, items in data.iteritems():
        tag_stats[str(label)][key] += 1

In [ ]:
tag_stats["0"] = defaultdict(int)
tag_stats["1"] = defaultdict(int)

In [86]:
data = defaultdict(dict)

In [87]:
i_from=0

with open(DATA_FILE) as df:
    for i, line in enumerate(df):
        if i < i_from:
            continue
            
        line = line.strip()
        parts = line.split()
        
        idx, mark, pageInb64, url = parts[0], int(parts[1]), parts[3], parts[2]
        html = base64.b64decode(pageInb64).decode('utf-8')
        
#         print i, mark
        
        html_de = HTMLDataExtractor()
        html_data = html_de.extract(html)
        
        html_fe = HTMLFeatureExtractor()
        html_fe.extract(html_data)
#         for key, value in html_fe.data.items(): 
#             print key
#             for k, v in value.items():
#                 if k == "bodies":
#                     pass
# #                     for b in v:
# #                         print b
#                 else:
#                     print "   ", k, v
        data[i]["features"] = html_fe.features
        data[i]["url"] = url
        data[i]["mark"] = mark
            
        
        
        
        
#         collect_tags_stats(mark, html_fe.data)
        
        
        sys.stdout.write('\r' + "%s" % (i))
        sys.stdout.flush()

#         if i - i_from >= 10:
#             break

7043

In [88]:
import codecs

In [89]:
import json
with codecs.open('result_without_stemming.json', 'w', 'utf-8') as fp:
    json.dump(data, fp)

In [90]:
from sklearn.feature_extraction import DictVectorizer

In [91]:
Y = []
D = []
URLS = []
for val in data.values():
    D.append(val["features"])
    URLS.append(val["url"])
    Y.append(val["mark"])

In [92]:
v = DictVectorizer(sparse=False)
X = v.fit_transform(D)


In [93]:
import numpy as np

In [94]:
np.savez("extracted_features_without_stemming.file", x = X, y=np.array(Y), f_names = np.array(v.feature_names_))

In [95]:
v.feature_names_

[u'a_count',
 u'a_mean_words_count',
 u'a_ratio_of_total_text_to_compressed',
 u'a_ratio_of_total_to_unique_words_counts',
 u'abbr_count',
 u'abbr_mean_words_count',
 u'abbr_ratio_of_total_text_to_compressed',
 u'abbr_ratio_of_total_to_unique_words_counts',
 u'b_count',
 u'b_mean_words_count',
 u'b_ratio_of_total_text_to_compressed',
 u'b_ratio_of_total_to_unique_words_counts',
 u'big_count',
 u'big_mean_words_count',
 u'big_ratio_of_total_text_to_compressed',
 u'big_ratio_of_total_to_unique_words_counts',
 u'blockquote_count',
 u'blockquote_mean_words_count',
 u'blockquote_ratio_of_total_text_to_compressed',
 u'blockquote_ratio_of_total_to_unique_words_counts',
 u'cite_count',
 u'cite_mean_words_count',
 u'cite_ratio_of_total_text_to_compressed',
 u'cite_ratio_of_total_to_unique_words_counts',
 u'code_count',
 u'code_mean_words_count',
 u'code_ratio_of_total_text_to_compressed',
 u'code_ratio_of_total_to_unique_words_counts',
 u'dfn_count',
 u'dfn_mean_words_count',
 u'dfn_ratio_of_to

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import numpy as np

In [ ]:
common_tags = np.array(list(set(tag_stats["0"].keys()) & set(tag_stats["1"].keys())))

In [ ]:
common_tags

In [ ]:
spam = []
nonspam = []
for tag in common_tags:
    spam.append(tag_stats["1"][tag])
    nonspam.append(tag_stats["0"][tag])

In [ ]:
spam = np.array(spam)
nonspam = np.array(nonspam)

In [ ]:
ratio = np.array(spam.astype(np.float) / (nonspam.astype(np.float) + 0.0001))

In [ ]:
sort_idx = np.argsort(ratio)

In [ ]:
plt.figure(figsize=(20, 20))

ind = np.arange(len(common_tags))
width = 1.0

p1 = plt.barh(ind, spam[sort_idx], width, color='r')
p2 = plt.barh(ind, nonspam[sort_idx], width, color='y', left=spam[sort_idx])


plt.ylabel('Counts')
plt.title('Tag counts')
plt.yticks(ind + width/2., ["%s(%2.2f)" % (common_tags[sort_idx][i], ratio[sort_idx][i]) for i in range(len(common_tags))]);

In [ ]:
common_tags[sort_idx][ratio[sort_idx] >= 1.30]

In [ ]:
plt.figure(figsize=(20, 15))

ind = np.arange(len(common_tags))
width = 0.9

p1 = plt.bar(ind, spam, width, color='r')
p2 = plt.bar(ind, nonspam, width, color='y', bottom=spam)

plt.ylabel('Counts')
plt.title('Tag counts')
plt.xticks(ind + width/2., common_tags,  rotation='vertical');

In [ ]:
for i, r in enumerate(ratio):
    if r < 0.4:
        print r, common_tags[i]

In [ ]:
for i, r in enumerate(ratio):
    if r > 1.6:
        print r, common_tags[i]